In [1]:
from src.gmail import Gmail
from src.newsletters.config import NEWSLETTER_AND_PARSER
from src.newsletters.parser.tldr import tldr_parser
from src.newsletters.parser.alpha_signal import alpha_signal_parser
from src.config import SCOPES

In [2]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(name)s - %(levelname)s -- l.%(lineno)d: %(message)s",
)

In [3]:
NEWSLETTER_AND_PARSER.keys()

dict_keys(['TLDR AI <dan@tldrnewsletter.com>', 'AlphaSignal <news@alphasignal.ai>', 'TLDR <dan@tldrnewsletter.com>', 'TLDR Product <dan@tldrnewsletter.com>'])

In [4]:
gmail = Gmail(
    path_to_token="/home/secrets_vault/token.json",
    path_to_credentials="/home/secrets_vault/credentials.json",
    scopes=SCOPES
)

2025-02-09 23:44:38,292 - googleapiclient.discovery_cache - INFO -- l.49: file_cache is only supported with oauth2client<4.0.0


# ALPHA SIGNAL

In [5]:
emails = gmail.fetch_emails(sender='AlphaSignal <news@alphasignal.ai>', after='2025-02-07', before='2025-02-07')
len(emails), type(emails)

2025-02-09 23:44:38,309 - src.gmail - DEBUG -- l.44: Fetch email for sender AlphaSignal <news@alphasignal.ai> from 2025-02-07 until 2025-02-07.
2025-02-09 23:44:38,321 - googleapiclient.discovery - DEBUG -- l.1258: URL being requested: GET https://gmail.googleapis.com/gmail/v1/users/me/messages?q=after%3A1738904400+before%3A1738990799+from%3AAlphaSignal+%3Cnews%40alphasignal.ai%3E&alt=json
2025-02-09 23:44:38,324 - google_auth_httplib2 - DEBUG -- l.118: Making request: POST https://oauth2.googleapis.com/token
2025-02-09 23:44:38,676 - googleapiclient.discovery - DEBUG -- l.1258: URL being requested: GET https://gmail.googleapis.com/gmail/v1/users/me/messages/194e1dae7e6bbcc6?alt=json


(1, list)

In [6]:
news_sources = alpha_signal_parser(emails[0])

2025-02-09 23:44:38,815 - src.newsletters.parser.alpha_signal - INFO -- l.34: Parsing email '🔥Mistral AI updates Chatbot, Faster than ChatGPT & Claude'
2025-02-09 23:44:38,932 - src.genai_model.genai_model - INFO -- l.106: Found 12 models for model_type small
2025-02-09 23:44:38,933 - src.genai_model.genai_model - INFO -- l.109: List of models included: ['gemini/gemini-1.5-flash-8b-latest', 'gemini/gemini-1.5-flash-8b-001', 'gemini/gemini-1.5-flash-8b-exp-0924', 'gemini/gemini-1.5-flash-8b-exp-0827', 'openrouter/google/gemini-flash-1.5-8b-exp', 'groq/gemma2-9b-it', 'openrouter/google/gemma-2-9b-it:free', 'mistal/ministral-8b-2410', 'groq/llama-3.1-8b-instant', 'openrouter/meta-llama/llama-3.1-8b-instruct:free', 'groq/llama-3-8b-8192', 'openrouter/meta-llama/llama-3-8b-instruct:free']
23:44:38 - LiteLLM:DEBUG: utils.py:299 - 

2025-02-09 23:44:38,934 - LiteLLM - DEBUG -- l.299: 

23:44:38 - LiteLLM:DEBUG: utils.py:299 - Request to litellm:
2025-02-09 23:44:38,937 - LiteLLM - DEBUG -- l.

In [7]:
news_sources

[{'title': 'Mistral AI brings Le Chat to iOS, Android, and enterprise',
  'url': 'https://mistral.ai/en/news/all-new-le-chat',
  'source_of_the_news': 'AlphaSignal <news@alphasignal.ai>',
  'text': 'French AI lab, Mistral AI updates its chatbot\xa0LE CHAT and makes it\r\navailable on iOS, Android, and enterprise infrastructure. It\r\nintroduces FLASH ANSWERS, A BUILT-IN CODE INTERPRETER, and REAL-TIME\r\nSEARCH, competing with ChatGPT, Claude, and DeepSeek.\r\n\r\nCORE FUNCTIONALITIES\r\n\r\nLe Chat provides tools for both general use and technical\r\napplications, offering:\r\n\r\n \t* OCR ENGINE: Processes PDFs, spreadsheets, and complex or\r\nlow-quality files.\r\n\r\n \t* CODE INTERPRETER: Runs scripts, handles data analysis, and creates\r\nvisualizations.\r\n\r\n \t* IMAGE GENERATION: Uses BLACK FOREST LABS’ FLUX ULTRA to generate\r\nphotorealistic visuals.\r\n\r\n \t* REAL-TIME INFORMATION RETRIEVAL: Combines static pre-trained\r\nknowledge with updated data from web searches and

In [23]:
news_sources[0]['url']

'https://mistral.ai/en/news/all-new-le-chat'

In [32]:
import pandas as pd
df = pd.read_parquet("/home/logs/database_news_stories.parquet")

In [33]:
df.dtypes

title                                           object
url                                             object
news_provider                                   object
source_of_the_news                              object
text                                            object
news_summary                                    object
date_source                 datetime64[ns, US/Eastern]
date_source_time_zone                           object
version                                         object
competitive_intelligence                        object
themes                                          object
market_intelligence                             object
personalities                                   object
score_category_count                             int64
unique_id                                        int64
dtype: object

## Experimentation

In [6]:
emails[0].keys()

dict_keys(['sender', 'subject', 'date_utc', 'id', 'text'])

In [7]:
emails[0]['sender']

'AlphaSignal <news@alphasignal.ai>'

In [8]:
emails[0]['subject']

'🔥Mistral AI updates Chatbot, Faster than ChatGPT & Claude'

In [9]:
print(emails[0]['text'])

On Andrew Ng's agentic object detection, GitHub Copilot's agent mode,
Google's Imagen 3 on Gemini API,... 

SIGNUP [1]  |  WORK WITH US [2]  |  FOLLOW ON X
[3]  |  READ ON WEB [4]

 . 

HEY ,

Welcome to AlphaSignal – the most read newsletter by AI
developers. 

We bring you the top 1% of news, papers, models, and repos, all
summarized to keep you updated on the latest in AI.

IN TODAY'S SIGNAL

_Read time: 5 min 41 sec_

🎖️ TOP NEWS

 	*
Mistral AI brings Le Chat to iOS, Android, and enterprise [5] with web
search and automation tools.

⚡️ TRENDING SIGNALS

 	*
Andrew Ng presents Agentic Object Detection [6]: human-like object
recognition via text-driven reasoning.

 	*
GitHub upgrades Copilot with agent mode [7], multi-file edits, and an
upcoming autonomous coding agent.

 	*
Google releases Imagen 3 in Gemini API [8]: high quality, watermarked
image generation with improved prompt adherence.

 	*
OpenAI refines CoT reasoning in O3-Mini [9] models, expanding
visibility.

 	*
Pika Lab

In [13]:
SECTIONS = [
    "TOP NEWS",
    "TRENDING SIGNALS",
    "TOP PAPERS",
    "PYTHON TIP",
]
txt = emails[0]['text']
sections = []
for sec in SECTIONS:
    _tmp = txt.split(sec)
    sections.append(_tmp[0])
    txt = _tmp[1]

In [15]:
for pp in sections[1:]:
    print(pp)



 	*
Mistral AI brings Le Chat to iOS, Android, and enterprise [5] with web
search and automation tools.

⚡️ 


 	*
Andrew Ng presents Agentic Object Detection [6]: human-like object
recognition via text-driven reasoning.

 	*
GitHub upgrades Copilot with agent mode [7], multi-file edits, and an
upcoming autonomous coding agent.

 	*
Google releases Imagen 3 in Gemini API [8]: high quality, watermarked
image generation with improved prompt adherence.

 	*
OpenAI refines CoT reasoning in O3-Mini [9] models, expanding
visibility.

 	*
Pika Labs launches Pikadditions [10]: add any object or person to any
video with ease.

💻


 	*
Google AI's AlphaGeometry2 surpasses gold medalists [11] in Olympiad
geometry, improving problem coverage and solving rates.

 	*
s1-32B improves test-time scaling [12] with budget forcing, boosting
math reasoning and competition scores.

 	*
Multi-agent system reduces AI hallucinations [13]using structured
reviews, NLP-based coordination, and new KPIs.

🧠 


In [19]:
print(emails[0]['text'].split("TOP NEWS")[2].split("WHAT'S NEW")[1].split("TRENDING SIGNALS")[0].split("TRY NOW")[0])



French AI lab, Mistral AI updates its chatbot LE CHAT and makes it
available on iOS, Android, and enterprise infrastructure. It
introduces FLASH ANSWERS, A BUILT-IN CODE INTERPRETER, and REAL-TIME
SEARCH, competing with ChatGPT, Claude, and DeepSeek.

CORE FUNCTIONALITIES

Le Chat provides tools for both general use and technical
applications, offering:

 	* OCR ENGINE: Processes PDFs, spreadsheets, and complex or
low-quality files.

 	* CODE INTERPRETER: Runs scripts, handles data analysis, and creates
visualizations.

 	* IMAGE GENERATION: Uses BLACK FOREST LABS’ FLUX ULTRA to generate
photorealistic visuals.

 	* REAL-TIME INFORMATION RETRIEVAL: Combines static pre-trained
knowledge with updated data from web searches and news outlets.

 	* Its MULTI-STEP AGENT FRAMEWORK lets you AUTOMATE WORKFLOWS by
integrating with messaging systems and databases.

PERFORMANCE

Le Chat runs on Mistral’s latest models and introduces FLASH
ANSWERS, a feature enabling RESPONSE SPEEDS OF UP TO 1,00

# TLDR

In [4]:
emails = gmail.fetch_emails(sender='TLDR AI <dan@tldrnewsletter.com>', after='2024-12-01', before='2024-12-07')
len(emails)

5

In [5]:
print(emails[0])

{'sender': 'TLDR AI <dan@tldrnewsletter.com>', 'subject': "OpenAI o1 System Card 📇, Perplexity Publisher Program 🔍, DeepMind's Genie 2 🧞", 'date_utc': 'Fri, 6 Dec 2024 14:16:24 +0000', 'id': '1939c5434a9f7fd8', 'text': "This report outlines the safety work carried out prior to releasing\r\nOpenAI o1 and o1-mini, including external red teaming and frontier\r\nrisk\r\nevaluation\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\r\n\r\n\r\n Sign Up [1] |Advertise [2]|View Online [3] \r\n\r\n\t\tTLDR \r\n\r\n\t\tTOGETHER

In [6]:
news_sources = tldr_parser(emails[0])

In [7]:
news_sources

[{'title': 'OPENAI O1 SYSTEM CARD',
  'url': 'https://openai.com/index/openai-o1-system-card/',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': "This report outlines the safety work carried out prior to releasing OpenAI o1 and o1-mini, including external red teaming and frontier risk evaluations according to OpenAI's Preparedness Framework.",
  'date_source': 'Fri, 6 Dec 2024 14:16:24 +0000',
  'new_provider': 'openai.com'},
 {'title': "PERPLEXITY EXPANDING IT'S PUBLISHER'S PROGRAM",
  'url': 'https://www.perplexity.ai/hub/blog/perplexity-expands-publisher-program-with-15-new-media-partners',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': "Perplexity has added over a dozen international news organizations to its Publishers' Program, offering tools, revenue sharing, and support to strengthen collaboration with global media.",
  'date_source': 'Fri, 6 Dec 2024 14:16:24 +0000',
  'new_provider': 'www.perplexity.ai'},
 {'title': 

In [8]:
links = emails[0]['text'].split("\r\n\r\n")[-1].split('\r\n')
links

['Links:',
 '------',
 '[1] https://tldr.tech/ai?utm_source=tldrai',
 '[2] https://advertise.tldr.tech/?utm_source=tldrai&utm_medium=newsletter&utm_campaign=advertisetopnav',
 '[3] https://a.tldrnewsletter.com/web-version?ep=1&lc=57aff87c-ae02-11ef-9074-4b7e3c69f0c6&p=68d03084-b3c7-11ef-928d-09d27092d252&pt=campaign&t=1733494584&s=a94ebb372672f7844f8d11fcad3029c68791b13faf831d296ca3c8f639256320',
 '[4] https://writer.com/product/ai-studio/?utm_source=tldr&utm_medium=newsletter&utm_campaign=ai_studio',
 '[5] https://links.tldrnewsletter.com/xS4pwo',
 '[6] https://links.tldrnewsletter.com/UUkpwY',
 '[7] https://techcrunch.com/2024/12/04/deepminds-genie-2-can-generate-interactive-worlds-that-look-like-video-games/?utm_source=tldrai',
 '[8] https://arxiv.org/abs/2412.03555?utm_source=tldrai',
 '[9] https://arxiv.org/abs/2412.02044v1?utm_source=tldrai',
 '[10] https://arxiv.org/abs/2412.03248v1?utm_source=tldrai',
 '[11] https://page.camunda.com/wp-rethinking-change-management?utm_medium=pa

In [14]:
"tldr" in links[6]

True

In [9]:
links[10].split('] ')

['[9', 'https://arxiv.org/abs/2412.02044v1?utm_source=tldrai']

In [10]:
type(news_sources[0]['url']), news_sources[0]['url']

(str, 'https://openai.com/index/openai-o1-system-card/')